# Post-training pruning: iterative magnitude pruning (IMP)

## When iteration is one, IMP downgrades to one-shot magnitude pruning.

In [1]:
import sys
sys.path.append('..')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse
from copy import deepcopy

from models import *
from utils.utils import progress_bar, train, test
from utils.pruner import pruning_model_random, check_sparsity, pruning_model, prune_model_custom, extract_mask, remove_prune

In [2]:
lr = 0.1
epochs = 2 # how many epochs for each training period?
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# 1 if OMP is applied. However, to get a fully retrained OMP model, pruning_times should be set to 2. 
# The model before the last pruning step is the fully trained OMP model given the desired pruning ratio.
pruning_times = 2
pruning_ratio = 0.2
rewinding_epoch = 1

save_dir = 'checkpoint'

In [3]:
# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# Model
print('==> Building model..')
net = ResNet18()
net = net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

print('######################################## Start Standard Training Iterative Pruning ########################################')

for state in range(pruning_times):

    print('******************************************')
    print('pruning state', state)
    print('******************************************')

    check_sparsity(net)

    for epoch in range(start_epoch, start_epoch+epochs):
        train(epoch, device, net, trainloader, optimizer, criterion)
        if state == 0:
            if (epoch+1) == rewinding_epoch:
                torch.save(net.state_dict(), os.path.join(save_dir, 'epoch_{}_rewind_weight.pt'.format(epoch+1)))
                rewind_init = deepcopy(net.state_dict())
        test(epoch, device, net, testloader, criterion, best_acc)
        scheduler.step()
    
    # model pruning and rewinding
    pruning_model(net, pruning_ratio)
    current_mask = extract_mask(net.state_dict())
    remove_prune(net)

    net.load_state_dict(rewind_init, strict=True)
    prune_model_custom(net, current_mask)
    
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    for _ in range(rewinding_epoch):
        scheduler.step()
    
check_sparsity(net)
print("Finished!")

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified
==> Building model..
######################################## Start Standard Training Iterative Pruning ########################################
******************************************
pruning state 0
******************************************
* remain weight =  100.0 %

Epoch: 0
 [========================>]  Step: 0ms | Tot: 0ms | Loss: 1.965 | Acc: 30.282% (15141/5000 391/391 
 [========================>]  Step: 0ms | Tot: 0ms | Loss: 1.668 | Acc: 40.640% (4064/1000 100/100 ==========>............]  Step: 0ms | Tot: 0ms | Loss: 1.668 | Acc: 41.020% (2051/500 50/100 
Saving..

Epoch: 1
 [========================>]  Step: 0ms | Tot: 0ms | Loss: 1.422 | Acc: 47.748% (23874/5000 391/391 
 [========================>]  Step: 0ms | Tot: 0ms | Loss: 1.339 | Acc: 51.820% (5182/1000 100/100 
Saving..
start unstructured pruning
remove pruning
start unstructured pruning with custom mask
**********

/localscratch/yaoyugua/miniconda3/envs/pruning/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


 [========================>]  Step: 0ms | Tot: 0ms | Loss: 1.384 | Acc: 49.512% (24756/5000 391/391 
 [========================>]  Step: 0ms | Tot: 0ms | Loss: 1.254 | Acc: 53.620% (5362/1000 100/100 =======>...............]  Step: 0ms | Tot: 0ms | Loss: 1.248 | Acc: 53.568% (1982/370 37/100 =================>.......]  Step: 0ms | Tot: 0ms | Loss: 1.258 | Acc: 53.639% (3862/720 72/100 
Saving..

Epoch: 1
 [========================>]  Step: 0ms | Tot: 0ms | Loss: 1.235 | Acc: 55.338% (27669/5000 391/391 
 [========================>]  Step: 0ms | Tot: 0ms | Loss: 1.224 | Acc: 55.330% (5533/1000 100/100 =======>...............]  Step: 0ms | Tot: 0ms | Loss: 1.216 | Acc: 55.462% (2163/390 39/100 
Saving..
start unstructured pruning
remove pruning
start unstructured pruning with custom mask
* remain weight =  64.00000465981887 %
Finished!
